In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/MLGSD/project_2/project_2'    #project folder directory

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd 'gdrive/My Drive/MLGSD/project_2/project_2'

/content/gdrive/My Drive/MLGSD/project_2/project_2


In [3]:
%load_ext autoreload
%autoreload 2
import torch
from utils import get_mnist_data
from models import ConvNN
from training_and_evaluation import evaluate_robustness_smoothing

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Project 2, part 4: Randomized smoothing certification (20 pt)
In this notebook we compare the robustness of the classifiers from Parts 1-3 via randomized smoothing.

## Your task
Complete the missing code in the respective files, i.e. `models.py`, `training_and_evaluation.py`, `attacks.py`, and this notebook. Make sure that all the functions follow the provided specification, i.e. the output of the function exactly matches the description in the docstring. 

Specifically, for this part you will have to implement the following functions / classes:  
**`training_and_evaluation.py`**:
* `evaluate_robustness_smoothing` (20pt)

## General remarks

Do not add or modify any code outside of the following comment blocks, or where otherwise explicitly stated.
``` python
##########################################################
# YOUR CODE HERE
...
##########################################################
```
After you fill in all the missing code, restart the kernel and re-run all the cells in the notebook.

The following things are **NOT** allowed:
- Using additional `import` statements
- Copying / reusing code from other sources (e.g. code by other students)

If you plagiarise even for a single project task, you won't be eligible for the bonus this semester.

In [0]:
mnist_testset = get_mnist_data(train=False)
use_cuda = torch.cuda.is_available()
model = ConvNN()
if use_cuda:
    model = model.cuda()
    
num_samples_1 = int(1e2)  # reduce this to 1e2 in case it takes too long, e.g. because you don't have CUDA
num_samples_2 = int(1e3)  # reduce this to 1e3 in case it takes too long, e.g. because you don't have CUDA
certification_batch_size = int(5e2)  # reduce this to 5e2 if required (e.g. not enough memory)
sigma = 1
alpha = 0.05

In [0]:
training_types = ['standard_training', "adversarial_training", "randomized_smoothing"]

### Robustness certification
Here we first load the checkpoints for the base classifiers of the different training methods of Parts 1-3. Then, perform robustness certification of the smooth classifier via randomized smoothing. For this, you need to implement `evaluate_robustness_smoothing` from `training_and_evaluation.py`. Follow the docstring in that file.

In [11]:
results = {}

for training_type in training_types:
    model.load_state_dict(torch.load(f"models/{training_type}.checkpoint"))
    certification_results = evaluate_robustness_smoothing(model, sigma, mnist_testset, num_samples_1=num_samples_1,
                                                          num_samples_2=num_samples_2, alpha=alpha, 
                                                          certification_batch_size=certification_batch_size)
    results[training_type] = certification_results

### Robustness comparison
Compare the robustness of the different training types. As we can see, robust training via randomized smoothing leads to the best robustness.

Note that the number of certified points will be lower in case you had to reduce the number of samples for performance reasons.

In [0]:
for k,v in results.items():
    print(f"{k}: correct_certified {v['correct_certified']}, avg. certifiable radius: {v['avg_radius']}")

standard_training: correct_certified 2087, avg. certifiable radius: 0.13772809505462646
adversarial_training: correct_certified 2408, avg. certifiable radius: 0.17466765642166138
randomized_smoothing: correct_certified 7707, avg. certifiable radius: 0.7908596396446228


In [12]:
for k,v in results.items():             #num_samples_1 100, num_samples_2 1000, certification_batch_size = 500
    print(f"{k}: correct_certified {v['correct_certified']}, avg. certifiable radius: {v['avg_radius']}")

standard_training: correct_certified 981, avg. certifiable radius: 0.1863873234622224
adversarial_training: correct_certified 1475, avg. certifiable radius: 0.23853548658882884
randomized_smoothing: correct_certified 7508, avg. certifiable radius: 0.905049787152439
